In [1]:
import agate

In [4]:
tester = agate.TypeTester(force = {
        'NrdPermit': agate.Text(),
        'PostalCD': agate.Text()
    }, limit=100)
wells = agate.Table.from_csv('wells.csv', column_types=tester)

In [5]:
print(wells)

|---------------+---------------|
|  column_names | column_types  |
|---------------+---------------|
|  X            | Number        |
|  Y            | Number        |
|  WellID       | Number        |
|  RegCD        | Text          |
|  Replacemen   | Number        |
|  Status       | Text          |
|  Useid        | Text          |
|  NrdName      | Text          |
|  NrdID        | Number        |
|  Countyname   | Text          |
|  CountyID     | Number        |
|  Township     | Number        |
|  Range        | Number        |
|  RangeDir     | Text          |
|  Section      | Number        |
|  SubSection   | Text          |
|  FootageNS    | Number        |
|  FootageDir   | Text          |
|  FootageEW    | Number        |
|  FootageD_1   | Text          |
|  NrdPermit    | Text          |
|  Acres        | Number        |
|  SeriesType   | Text          |
|  SeriesEnd    | Date          |
|  PumpRate     | Number        |
|  PColDiam     | Number        |
|  PumpDepth  

In [10]:
oldest = wells.aggregate(agate.Min('Compdate'))

In [11]:
print(oldest)

1895-01-01


In [12]:
newest = wells.aggregate(agate.Max('Compdate'))

In [13]:
print(newest)

2016-02-25


In [46]:
class ActiveWell(agate.Computation):
    """
    Computes whether a well is active in a year
    """
    def __init__(self, column_name_start, column_name_end, year):
        self._column_start = column_name_start
        self._column_end = column_name_end
        self._year = year

    def get_computed_data_type(self, table):
        """
        The return value is a boolean.
        """
        return agate.Boolean()

    def validate(self, table):
        """
        Verify the column is date.
        """
        columnStart = table.columns[self._column_start]

        if not isinstance(columnStart.data_type, agate.Date):
            raise agate.DataTypeError('Can only be applied to date data.')
            
        columnEnd = table.columns[self._column_end]

        if not isinstance(columnEnd.data_type, agate.Date):
            raise agate.DataTypeError('Can only be applied to date data.')

    def run(self, table):
        new_column = []
        
        for row in table.rows:
            start_val = row[self._column_start]
            if start_val is not None:
                start_year = start_val.year
            else:
                start_year = 1700
            end_val = row[self._column_end]
            if end_val is not None:
                end_year = end_val.year
            else:
                end_year = 5000
            if start_year < self._year < end_year:
                status = True
                new_column.append(status)
            else:
                status = False
                new_column.append(status)
        return new_column

In [47]:
active_wells = wells.compute([
    ('2010active', ActiveWell('Compdate', 'DecommDate', 2010))
        ])

In [50]:
active_wells.print_table(10)

|-----------------------+----------------------+---------+----------+------------+--------+-------+----------------------+-------+--------------+----------+----------+-------+----------+---------+------------+-----------+------------+-----------+------------+-----------+-------+------------+------------+----------+----------+-----------+------------+-----+-----+----------+---------+----------------------+---------------------+----------------------+--------------+----------+----------+------------+------------+------------+------------+---------------+-----------------+---------+-------------|
|                     X |                    Y |  WellID | RegCD    | Replacemen | Status | Useid | NrdName              | NrdID | Countyname   | CountyID | Township | Range | RangeDir | Section | SubSection | FootageNS | FootageDir | FootageEW | FootageD_1 | NrdPermit | Acres | SeriesType |  SeriesEnd | PumpRate | PColDiam | PumpDepth | TotalDepth | SWL | PWL | CertifID | OwnerID | FirstName    